In [16]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

Cargar y limpiar datos

In [18]:
housing = pd.read_csv('housing.csv')
housing = pd.get_dummies(housing, drop_first=True)
housing['total_bedrooms'] = housing['total_bedrooms'].fillna(housing['total_bedrooms'].mean())
target = 'median_house_value'
X = housing.copy().drop(target, axis=1)
y = housing[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [1]:
# Ejempolo gradient boosting


In [2]:
## Cross validation 


grid_gbt.best_estimator_.named_steps['gbt']

In [3]:
# Mejor estimador


In [4]:
# r2 score


In [5]:
## Calibracion


In [6]:
# Entendiendo el modelo 


    PartialDependenceDisplay.from_estimator(
        best_grid_gbt,
        X_train,
        [i],
        feature_names=X_train.columns
    )

In [7]:
# Codigo for para partial dependence plot 
